## PYOMO 101

In [ ]:
import pyomo.environ as pyo

model = pyo.ConcreteModel()

print(vars(model))

model.x = pyo.Var([1,2], domain=pyo.NonNegativeReals)

model.OBJ = pyo.Objective(expr= 2*model.x[1] + 3*model.x[2])

model.Constraint1 = pyo.Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)

solver = pyo.SolverFactory("gurobi")
result = solver.solve(model)
result

## GurobiPy 

In [1]:
import gurobipy as gp
import logging 
import os 

# configure logging
log_dir = "logs"
log_file = "script_log.txt"

# ensure log directory exists
os.makedirs(log_dir, exist_ok=True)

logging.basicConfig(
    filename=os.path.join(log_dir, log_file),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

env = gp.Env(params={"LogToConsole": 0})

# Create a new model
m = gp.Model(env=env)

# Create variables
x = m.addVar(vtype='B', name="x")
y = m.addVar(vtype='B', name="y")
z = m.addVar(vtype='B', name="z")

# Set objective function
m.setObjective(x + y + 2 * z, gp.GRB.MAXIMIZE)

# Add constraints
m.addConstr(x + 2 * y + 3 * z <= 4)
m.addConstr(x + y >= 1)

# Solve it!
m.optimize()

print(f"Optimal objective value: {m.objVal}")
print(f"Solution values: x={x.X}, y={y.X}, z={z.X}")

Optimal objective value: 3.0
Solution values: x=1.0, y=0.0, z=1.0


In [3]:
print(gp.disposeDefaultEnv())

None


In [4]:
gp.paramHelp("Cuts")
gp.paramHelp("Heu*")
gp.paramHelp("*cuts")

Set parameter Username
Set parameter LicenseID to value 2654438
Academic license - for non-commercial use only - expires 2026-04-19

 PARAMETER NAME:
   Cuts

 TYPE:
   Integer

 PURPOSE:
   Global cut aggressiveness setting. Use value 0 to shut off cuts, 1 for
   moderate cut generation, 2 for aggressive cut generation, and 3 for
   very aggressive cut generation. The default -1 value chooses
   automatically. This parameter is overridden by the parameters that
   control individual cut types (e.g., CliqueCuts).

   Note:

     Only affects mixed integer programming (MIP) models

 VALUES:
   Minimum : -1
   Maximum : 3
   Default : -1


 PARAMETER NAME:
   Heuristics

 TYPE:
   Double

 PURPOSE:
   Determines the amount of time spent in MIP heuristics. You can think
   of the value as the desired fraction of total MIP runtime devoted to
   heuristics (so by default, we aim to spend 5% of runtime on
   heuristics). Larger values produce more and better feasible solutions,
   at a cost 

## Study line parameters 

In [1]:
import pandapower as pp 
import numpy as np 

net = pp.create_empty_network()

bus1 = pp.create_bus(net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")


# standard 20 kV line type 
std_line_type = "NAYY 4x50 SE"

# Create standard lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1.0,
               std_type=std_line_type, name="Line 1-2")

pp.create_ext_grid(net, bus=bus1, vm_pu=1.02, name="Grid Connection")

# pp.create_load(net, bus3, p_mw=0.2, q_mvar=0.05, name="Load Bus 3")
# pp.create_load(net, bus4, p_mw=0.3, q_mvar=0.1, name="Load Bus 4")

pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


In [2]:
# bus admittance for bus 1 - 2 
Ybus = net._ppc["internal"]["Ybus"].todense()
# Ybus[0,0]*Ybus[1,1] - Ybus[0,1]*Ybus[1,0]
Ybus

matrix([[ 612.81031278-79.2130607j , -612.81031278+79.22625539j],
        [-612.81031278+79.22625539j,  612.81031278-79.2130607j ]])

In [ ]:
sys_freq = 50 # [Hz]
# for lines

# convert r_ohm_per_km to r_ohm
r_ohm = (net.line['r_ohm_per_km'] * net.line['length_km'] / net.line['parallel']) 

# convert r_ohm_per_km to r_per_unit
r_pu = (net.line['r_ohm_per_km'] * net.line['length_km'] / net.line['parallel']) / (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values 

# convert x_ohm_per_km to x_ohm
x_ohm = (net.line['x_ohm_per_km'] * net.line['length_km'] / net.line['parallel']) 

# convert x_ohm_per_km to x_ohm
x_pu = (net.line['x_ohm_per_km'] * net.line['length_km'] / net.line['parallel']) / (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values


z_pu = r_pu + 1j*x_pu

z_ohm = r_ohm + 1j*x_ohm

y_series_pu = 1 / z_pu
y_series_mho = 1 / z_ohm

# convert b_ohm_per_km to b_mho
b_mho = ( 2 * np.pi * sys_freq * net.line['c_nf_per_km'] * 10**(-9) * net.line['length_km'] * net.line['parallel']) 

# convert c_nf_per_km to b_pu
b_pu = ( 2 * np.pi * sys_freq * net.line['c_nf_per_km'] * 10**(-9) * net.line['length_km'] * net.line['parallel']) * (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values

# convert r_ohm_per_km to g_mho
g_mho = ( net.line['g_us_per_km'] * 10**(-6) * net.line['length_km'] * net.line['parallel']) 

# convert g_us_per_km to g_pu
g_pu = ( 2 * np.pi * sys_freq * net.line['g_us_per_km'] * 10**(-6) * net.line['length_km'] * net.line['parallel']) * (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values 



y_sh_mho = g_mho - 1j*b_mho



a_1_pu = y_series_pu + y_sh_pu/2
a_2_pu = - y_series_pu
a_3_pu = - y_series_pu
a_4_pu = y_series_pu + y_sh_pu/2

a_1_mho = y_series_mho + y_sh_mho / 2
a_2_mho = - y_series_mho
a_3_mho = - y_series_mho
a_4_mho = y_series_mho + y_sh_mho / 2


def cal_ABCD(a_1, a_2, a_3, a_4): 
    A = -a_4/a_3 
    B = -1/a_3
    C = a_2 - a_1 * a_4 / a_3
    D = -a_1/a_3
    return A, B, C, D 

A_pu, B_pu, C_pu, D_pu = cal_ABCD(a_1_pu, a_2_pu, a_3_pu, a_4_pu)

A, B, C, D = cal_ABCD(Ybus[0,0], Ybus[0,1], Ybus[1,0], Ybus[1,1])

ABCD_conditio_pu = A_pu * D_pu - B_pu * C_pu 
ABCD_condition = A * D - B * C 

print(ABCD_condition)

# abcd_condition_pu_ii = a_1_pu * a_4_pu - a_2_pu * a_3_pu
# abcd_condition_mho_ii = a_1_mho * a_4_mho - a_2_mho * a_3_mho

# abcd_condition_pu = a_2_pu / a_3_pu - a_1_pu * a_4_pu / a_3_pu
# abcd_condition_mho = a_2_mho / a_3_mho - a_1_mho * a_4_mho / a_3_mho

# abcd_condition_pu_num = a_2_pu / a_2_pu - a_1_pu * a_1_pu / a_2_pu
# abcd_condition_mho_num = a_2_mho / a_2_mho - a_1_mho * a_1_mho / a_2_mho


# abcd_condition_pu, abcd_condition_mho, abcd_condition_pu_ii, abcd_condition_mho_ii, abcd_condition_pu_num, abcd_condition_mho_num
# net.line['a_1'] = y_series + y_sh/2
# net.line['a_2'] = - y_series
# net.line['a_3'] = - y_series
# net.line['a_4'] = y_series + y_sh/2

(1.0000000000000004+2.0274580625478933e-17j)


In [4]:
np.round(np.real_if_close(ABCD_conditio_pu.values),2) == 1.

array([ True])

In [5]:
a_1_pu, a_2_pu, a_3_pu, a_4_pu

(0    612.810313- 79.239450j
 dtype: complex128,
 0   -612.810313+ 79.226255j
 dtype: complex128,
 0   -612.810313+ 79.226255j
 dtype: complex128,
 0    612.810313- 79.239450j
 dtype: complex128)

In [5]:
y_series_pu[0]

np.complex128(612.8103127766655-79.22625539012964j)

## Study Transformer Parameters 

In [21]:
import pandapower as pp 
import numpy as np 

np.set_printoptions(precision=4)

net = pp.create_empty_network()


bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")  # connected via transformer


# standard 20 kV line type 
std_line_type = "NAYY 4x50 SE"


pp.create_transformer(net, bus2, bus3, std_type="0.4 MVA 20/0.4 kV", name="Trafo 2-3")
net.trafo.shift_degree = 0.0 # bug in pandapwer 

pp.create_ext_grid(net, bus=bus2, vm_pu=1.02, name="Grid Connection")


pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


In [22]:
# bus admittance for bus 1 - 2 
Ybus = np.array(net._ppc["internal"]["Ybus"].todense())
# Ybus[0,0]*Ybus[1,1] - Ybus[0,1]*Ybus[1,0]
Ybus

array([[ 1.5837-6.4759j, -1.583 +6.4759j],
       [-1.583 +6.4759j,  1.5837-6.4759j]])

In [23]:
# 1. series losses: joule or heat losses
# short-circuit resistance r
r_s_trafo = (net.trafo['vkr_percent']/100 * net.sn_mva / net.trafo['sn_mva']) 

# short-circuit impedance magnitude
z_s_trafo = (net.trafo['vk_percent']/100 * net.sn_mva / net.trafo['sn_mva'])

# short-circuit reactance 
x_s_trafo = np.sqrt(z_s_trafo**2 - r_s_trafo**2)    

# series impedance 
z_s_trafo_j = r_s_trafo + 1j*x_s_trafo

# series admittance 
y_s_trafo = 1 / z_s_trafo_j

# 2. magnetising losses: eddy currents and hysterisis 
# magnetising conductance g
g_sh_trafo = (net.trafo['pfe_kw']/(net.trafo['sn_mva'] * 1000) * net.sn_mva / net.trafo['sn_mva'])

# shunt admittance 
y_sh_trafo = (net.trafo['i0_percent']/100) * (net.sn_mva / net.trafo['sn_mva']) 


# magnetising impedance b
b_sh_trafo = np.sqrt(y_sh_trafo**2- g_sh_trafo**2)

y_sh_trafo_j = g_sh_trafo + 1j*b_sh_trafo

# pandapower formula
phase_shift = 0.0
tap_nom = 1 + (net.trafo.tap_pos - net.trafo.tap_neutral) * (net.trafo.tap_step_percent / 100) # this is tau!
N_tap = tap_nom * np.exp(1j*phase_shift)

a_1 = (y_s_trafo + y_sh_trafo_j/2) * 1 / (N_tap * np.conj(N_tap))
a_2 = -y_s_trafo * 1 / (np.conj(N_tap))
a_3 = -y_s_trafo * 1 / (N_tap)
a_4 = y_s_trafo + y_sh_trafo_j/2

Y_bus_equations = np.array([[a_1, a_2],
                            [a_3, a_4]])


In [24]:
Ybus = Ybus.reshape(2,2)
Y_bus_equations = Y_bus_equations.reshape(2,2)

In [25]:
Ybus - Y_bus_equations

array([[-0.0039-1.6603e-08j,  0.0003-1.6603e-08j],
       [ 0.0003-1.6603e-08j, -0.0039-1.6603e-08j]])

In [26]:
ps_s = net.trafo.shift_degree * np.pi/180
net.trafo.loc[:,"shift_rad"] = ps_s
tap_nom_s = 1 + (net.trafo.tap_pos - net.trafo.tap_neutral) * (net.trafo.tap_step_percent / 100)
net.trafo.loc[:,"tap_nom"] = tap_nom_s 
N_tap_s = tap_nom_s * np.exp(1j*ps_s)

for id, row in net.trafo.iterrows(): 
    a_1t = Ybus[row.hv_bus, row.hv_bus]
    a_2t = Ybus[row.hv_bus, row.lv_bus]
    a_3t = Ybus[row.lv_bus, row.hv_bus]
    a_4t = Ybus[row.lv_bus, row.lv_bus]

    # series impedance 
    y_series_trafo = - a_3t * N_tap_s[id] 

    # r, x 
    z_s_trafo = 1 / y_series_trafo
    r_trafo, x_trafo = np.real(z_s_trafo), np.imag(z_s_trafo)

    # net.trafo.loc[id, 'r'] = r_trafo
    # net.trafo.loc[id, 'x'] = x_trafo 

    # shunt impedance 
    y_sh_trafo = 2* (a_4t - y_series_trafo) 
    g_trafo, b_trafo = np.real(y_sh_trafo), np.imag(y_sh_trafo)

    # net.trafo.loc[id,'g'] = g_trafo 
    # net.trafo.loc[id,'b'] = b_trafo

In [31]:
y_s_trafo - y_series_trafo

0    0.000337-0.000000j
dtype: complex128

In [32]:
y_sh_trafo_j - y_sh_trafo

0    0.007088+0.000000j
dtype: complex128

In [33]:
g_sh_trafo - g_trafo

0    0.007088
dtype: float64

In [34]:
b_sh_trafo - b_trafo

0    6.641188e-08
dtype: float64

In [8]:
net = add_branch_parameters(net)

net.trafo

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_step_degree,tap_pos,tap_changer_type,id_characteristic_table,tap_dependency_table,parallel,df,in_service,shift_rad,tap_nom


In [35]:
import pandapower 

In [36]:
net = pp.create_empty_network()

bus1 = pp.create_bus(net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")  # connected via transformer
bus4 = pp.create_bus(net, vn_kv=20.0, name="Bus 4")

# standard 20 kV line type 
std_line_type = "NAYY 4x50 SE"

# Create standard lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1.0,
               std_type=std_line_type, name="Line 1-2")

pp.create_line(net, from_bus=bus1, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 1-4")

pp.create_line(net, from_bus=bus2, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 2-4")

pp.create_transformer(net, bus2, bus3, std_type="0.25 MVA 20/0.4 kV", name="Trafo 2-3")
net.trafo.shift_degree = 0.0 # bug in pandapwer 

pp.create_ext_grid(net, bus=bus2, vm_pu=1.02, name="Grid Connection")

pp.create_load(net, bus3, p_mw=0.2, q_mvar=0.05, name="Load Bus 3")
pp.create_load(net, bus4, p_mw=0.3, q_mvar=0.1, name="Load Bus 4")

pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


In [53]:
from_bus, to_bus = net.line.from_bus.values, net.line.to_bus.values
hv_bus, lv_bus = net.trafo.hv_bus.values, net.trafo.lv_bus.values

src_bus = np.concat([from_bus, hv_bus])
tgt_bus = np.concat([to_bus, lv_bus])

branch_tuple = [(int(u),int(v)) for u, v in zip(src_bus, tgt_bus)]
param_dict = {branch: [] for branch in branch_tuple}

line_tuple = [(int(u),int(v)) for u, v in zip(from_bus, to_bus)]
trafo_tuple = [(int(u),int(v)) for u, v in zip(hv_bus, lv_bus)]

In [55]:
line_tuple + trafo_tuple

[(0, 1), (0, 3), (1, 3), (1, 2)]

## Checking add_branch_parameters 

In [9]:
import pandapower 
import warnings 

In [10]:
def add_branch_parameters(net: pp.pandapowerNet): 
    """Calculates the branch parameters of the bus-branch model. 
    if branch is line, then net.line has added columns as r, x, b, g
    if branch is trafo, then net.trafo has added columns as r, x, b, g, tap, shift 
    
    Assumptions: 
    No mutual inductance between lines. 

    Note: 
    Pandapower equations from documentation for trafo r, x, b and g are ambiguous, and do not match with the 
    internal y-bus matrices. So, branch parameters for trafo are calculated from Y_bus internals and not 
    the pandapower equations. 

    Returns: 
    net: Pandapower Net with added branch parameter for line and trafo. 
    
    """
    # remove switches 
    net.switch.drop(net.switch.index, inplace = True)
    net.res_switch.drop(net.res_switch.index, inplace = True)

    sys_freq = 50 # [Hz]
    
    # check 1: if the required columns are present in lines dataframe  
    required_columns = {
        'line':['r_ohm_per_km', 'length_km', 'parallel', 'from_bus', 'x_ohm_per_km', 'c_nf_per_km', 'g_us_per_km'],
        'bus':['vn_kv'],
        'trafo': ['lv_bus', 'vn_lv_kv', 'sn_mva', 'vkr_percent', 'vk_percent', 'pfe_kw', 'i0_percent', 'hv_bus', 'vn_hv_kv']
    }

    for element, columns in required_columns.items(): 
        for column in columns:
            if column not in net[element].columns:
                warnings.warn(f"Column '{column}' is missing in '{element}', padding with zeros.")
                net[element][column] = 0    
    
    # line branch parameters 
    sys_freq = 50

    # convert r_ohm_per_km to r_pu
    r_pu = (net.line['r_ohm_per_km'] * net.line['length_km'] / net.line['parallel']) / (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values
    net.line.loc[:,'r_pu'] = r_pu

    # convert x_ohm_per_km to x_ohm
    x_pu = (net.line['x_ohm_per_km'] * net.line['length_km'] / net.line['parallel']) / (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values
    net.line.loc[:,'x_pu'] = x_pu

    z_pu = r_pu + 1j*x_pu
    y_series = 1 / z_pu

    # convert c_nf_per_km to b_mho
    b_pu = ( 2 * np.pi * sys_freq * net.line['c_nf_per_km'] * 10**(-9) * net.line['length_km'] * net.line['parallel']) * (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values 
    net.line.loc[:, 'b_pu'] = b_pu

    # convert g_us_per_km to g_mho
    g_pu = ( 2 * np.pi * sys_freq * net.line['g_us_per_km'] * 10**(-6) * net.line['length_km'] * net.line['parallel']) * (net.bus['vn_kv'].loc[net.line['from_bus'].values]**2/net.sn_mva).values
    net.line.loc[:, 'g_pu'] = g_pu

    # add zeros for tap and shift degree in lines 
    net.line.loc[:, 'tap_nom'] = 0 
    net.line.loc[:, 'shift_rad'] = 0


    y_sh = g_pu - 1j*b_pu    

    # get Y-bus for lines only 
    a_1 = y_series + y_sh/2
    a_2 = - y_series
    a_3 = - y_series
    a_4 = y_series + y_sh/2

    yb_size = len(net.bus)

    Ybline = np.zeros((yb_size, yb_size)).astype(complex)

    fb_line = net.line.from_bus 
    tb_line = net.line.to_bus 

    line_idx = net.line.index 

    for (idx, fb, tb) in zip(line_idx, fb_line, tb_line): 
        Ybline[fb, fb] = complex(a_1[idx])
        Ybline[fb, tb] = complex(a_2[idx])
        Ybline[tb, fb] = complex(a_3[idx])
        Ybline[tb, tb] = complex(a_4[idx])

    # get the pandpaower internal YBus 
    pp.runpp(net)
    Ybus = np.array(net._ppc["internal"]["Ybus"].todense())

    Ybus_trafo = Ybus - Ybline 

    if sum(net.trafo.tap_pos.isna()) > 0: 
        print("Filling nan as 0 in tap_pos, tap_neutral, tap_step_degree")
        net.trafo.loc[net.trafo.loc[:,'tap_pos'].isna(),'tap_pos'] = 0
        net.trafo.loc[net.trafo.loc[:,'tap_neutral'].isna(),'tap_neutral'] = 0
        net.trafo.loc[net.trafo.loc[:,'tap_step_degree'].isna(),'tap_step_degree'] = 0
        
        

    ps_s = net.trafo.shift_degree * np.pi/180
    net.trafo.loc[:,"shift_rad"] = ps_s
    tap_nom_s = 1 + (net.trafo.tap_pos - net.trafo.tap_neutral) * (net.trafo.tap_step_percent / 100)
    net.trafo.loc[:,"tap_nom"] = tap_nom_s 
    N_tap_s = tap_nom_s * np.exp(1j*ps_s)


    for id, row in net.trafo.iterrows(): 
        a_1t = Ybus_trafo[row.hv_bus, row.hv_bus]
        a_2t = Ybus_trafo[row.hv_bus, row.lv_bus]
        a_3t = Ybus_trafo[row.lv_bus, row.hv_bus]
        a_4t = Ybus_trafo[row.lv_bus, row.lv_bus]

        # series impedance 
        y_series_trafo = - a_3t * N_tap_s[id] 

        # r, x 
        z_s_trafo = 1 / y_series_trafo
        r_trafo, x_trafo = np.real(z_s_trafo), np.imag(z_s_trafo)

        net.trafo.loc[id, 'r'] = r_trafo
        net.trafo.loc[id, 'x'] = x_trafo 

        # shunt impedance 
        y_sh_trafo = 2* (a_4t - y_series_trafo) 
        g_trafo, b_trafo = np.real(y_sh_trafo), np.imag(y_sh_trafo)

        net.trafo.loc[id,'g'] = g_trafo 
        net.trafo.loc[id,'b'] = b_trafo
    
    return net


In [11]:
net = pp.create_empty_network()

bus1 = pp.create_bus(net, vn_kv=20.0, name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20.0, name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=0.4, name="Bus 3")  # connected via transformer
bus4 = pp.create_bus(net, vn_kv=20.0, name="Bus 4")

# standard 20 kV line type 
std_line_type = "NAYY 4x50 SE"

# Create standard lines
pp.create_line(net, from_bus=bus1, to_bus=bus2, length_km=1.0,
               std_type=std_line_type, name="Line 1-2")

pp.create_line(net, from_bus=bus1, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 1-4")

pp.create_line(net, from_bus=bus2, to_bus=bus4, length_km=1.0,
               std_type=std_line_type, name="Line 2-4")

pp.create_transformer(net, bus2, bus3, std_type="0.25 MVA 20/0.4 kV", name="Trafo 2-3")
net.trafo.shift_degree = 0.0 # bug in pandapwer 

pp.create_ext_grid(net, bus=bus2, vm_pu=1.02, name="Grid Connection")

pp.create_load(net, bus3, p_mw=0.2, q_mvar=0.05, name="Load Bus 3")
pp.create_load(net, bus4, p_mw=0.3, q_mvar=0.1, name="Load Bus 4")

pp.runpp(net)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


In [12]:
net = add_branch_parameters(net)

net.trafo

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_dependency_table,parallel,df,in_service,shift_rad,tap_nom,r,x,g,b
0,Trafo 2-3,0.25 MVA 20/0.4 kV,1,2,0.25,20.0,0.4,6.0,1.44,0.8,...,False,1,1.0,True,0.0,1.0,0.05759,0.232991,0.0008,-3.727682e-08


In [16]:
[*range(5)]

[0, 1, 2, 3, 4]

In [ ]:
# # how many buses are there? 
# num_buses = len(net.bus.index)

# # how many state-variables 
# n = 2*num_buses - 1 # \theta_1 = 0 for reference 

# # consider fully-observable network, so number of measurements >= number of states 
# # using two measurements of active power and voltage at each bus 
# m = 2*num_buses

# # initialize model 
# gp_model = gp.Model("WLS_State_Estimation")

# # measurement vector as 5% tolerance to power flow results 
# V_meas = np.random.normal(net.res_bus.vm_pu.values, 0.05/3)
# P_meas = np.random.normal(net.res_bus.p_mw.values, 0.05/3)

# # Gurobi variables for estimated voltages and angles
# # subject to voltage violations and angle violations 
# X_vm_pu = {i: gp_model.addVar(name=f"|V|_bus{i}") for i in range(num_buses)}
# X_va_rad = {i: gp_model.addVar(name=f"A_bus{i}") for i in range(num_buses)}

# # slack bus angle 
# X_va_rad[0].LB, X_va_rad[0].UB = 0.0, 0.0 
# gp_model.update()

# # get line and transformer parameters
# # param_dict = {(i, j): [G_ij, B_ij, g_s_ij, b_s_ij, g_sh_ij, b_sh_ij]}
# param_dict = get_branch_parameters(net=net) 


# # get neighbor_dictionary 
# # neighbor_dict = {src: [neighbors]}
# neighbor_dict = get_neighbor_dict(edge_index)

# # Gurobi variables for estimated voltages and angles
# # subject to voltage violations and angle violations 
# P_meas_expr = {int(i): [] for i in nodes}
# V_meas_expr = {int(i): [] for i in nodes}

# for i in nodes: 
#     i = int(i)
#     P_meas_i = 0.0
#     for j in neighbor_dict[i]:
#         j = int(j)
#         P_meas_i += X_vm_pu[i] * X_vm_pu[j] * (param_dict[(i,j)][0]*nlfunc.cos(X_va_rad[i] - X_va_rad[j]) + 
#                                                param_dict[(i,j)][1]*nlfunc.sin(X_va_rad[i] - X_va_rad[j]))
#     # capture the expressions 
#     P_meas_expr[i] = P_meas[i] + P_meas_i 
#     V_meas_expr[i] = X_vm_pu[i] - V_meas[i]


# objective = sum(P_meas_expr.values()) + sum(V_meas_expr.values())

# gp_model.setObjective(objective, GRB.MINIMIZE)